In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 1 #当日は0、前日は1・・・
date2 = "08/17"
pmc = "11002018" # 店舗ID
SPREADSHEET_KEY = '1ERAWxxUmiu-EkKFI-_MWCbc0FnxPMEJDswAX2Uoaq_4' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 1001 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+123@gmail.com"
user_pw = "paruparu21"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
chrome_service = ChromeService(executable_path='C:\\Users\\garden026\\WD\\114.0.5735.90\\chromedriver.exe')

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value
s2row = worksheet.cell(11,  30).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []
s2table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "500":
                    s5table.append(li)
                if unit == "200":
                    s2table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "500":
                        s5table.append(li)
                    if unit == "200":
                        s2table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})
wb.values_append("'S2RAW'!F" + str(int(s2row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s2table})

['08/17', '025837', '1', '1026.0', '-902.0', '0', '0']
['08/17', '025827', '2', '11026.0', '-9027.0', '0', '0']
['08/17', '025864', '3', '3590.0', '6019.0', '11', '1']
['08/17', '025852', '4', '0', '0', '0', '0']
['08/17', '025550', '5', '5385.0', '-2406.0', '1', '1']
['08/17', '025680', '6', '0', '0', '0', '0']
['08/17', '025859', '7', '1282.0', '-1203.0', '0', '0']
['08/17', '025859', '8', '3077.0', '-2707.0', '0', '0']
['08/17', '025806', '9', '0', '0', '0', '0']
['08/17', '025806', '10', '0', '0', '0', '0']
['08/17', '025789', '11', '0', '0', '0', '0']
['08/17', '025789', '12', '0', '0', '0', '0']
['08/17', '025789', '13', '0', '0', '0', '0']
['08/17', '025789', '14', '3333.0', '15649.0', '12', '1']
['08/17', '025606', '15', '3590.0', '-3008.0', '0', '0']
['08/17', '025606', '16', '1538.0', '6621.0', '5', '1']
['08/17', '025606', '17', '5128.0', '-4212.0', '0', '0']
['08/17', '025606', '18', '2051.0', '904.0', '3', '2']
['08/17', '025606', '19', '256.0', '-902.0', '0', '0']
['08/17

['08/17', '025761', '148', '0', '0', '0', '0']
['08/17', '025573', '149', '15385.0', '2107.0', '13', '3']
['08/17', '025754', '150', '0', '0', '0', '0']
['08/17', '025779', '151', '2051.0', '-1805.0', '0', '0']
['08/17', '025779', '152', '256.0', '1806.0', '3', '1']
['08/17', '025839', '153', '30256.0', '-12638.0', '13', '2']
['08/17', '025839', '154', '5897.0', '9330.0', '12', '1']
['08/17', '025842', '155', '0', '0', '0', '0']
['08/17', '025842', '156', '0', '0', '0', '0']
['08/17', '025687', '157', '4103.0', '-3610.0', '0', '0']
['08/17', '025687', '158', '4103.0', '-2406.0', '1', '1']
['08/17', '025886', '159', '11026.0', '-9629.0', '0', '0']
['08/17', '025886', '160', '3846.0', '-1203.0', '2', '1']
['08/17', '025856', '161', '21282.0', '904.0', '13', '6']
['08/17', '025856', '162', '9487.0', '7223.0', '10', '3']
['08/17', '025856', '163', '15128.0', '13242.0', '17', '3']
['08/17', '025856', '164', '13333.0', '-11133.0', '0', '0']
['08/17', '025856', '165', '8462.0', '-902.0', '5',

['08/17', '025859', '289', '21795.0', '-1805.0', '6', '1']
['08/17', '025364', '290', '48205.0', '-13541.0', '33', '18']
['08/17', '025364', '291', '33846.0', '-2106.0', '35', '12']
['08/17', '025364', '292', '33077.0', '4214.0', '44', '16']
['08/17', '025364', '293', '40513.0', '2107.0', '50', '20']
['08/17', '025364', '294', '43077.0', '6320.0', '60', '22']
['08/17', '025364', '295', '24359.0', '3913.0', '34', '14']
['08/17', '025364', '296', '33846.0', '-5115.0', '31', '10']
['08/17', '025364', '297', '37692.0', '10232.0', '61', '19']
['08/17', '025364', '298', '16154.0', '-9930.0', '3', '3']
['08/17', '025364', '299', '38462.0', '-1805.0', '39', '12']
['08/17', '025364', '300', '32564.0', '2408.0', '41', '13']
['08/17', '025364', '301', '32308.0', '-1203.0', '33', '14']
['08/17', '025364', '302', '32308.0', '6922.0', '46', '16']
['08/17', '025317', '303', '28718.0', '-3610.0', '28', '13']
['08/17', '025317', '304', '26154.0', '603.0', '29', '10']
['08/17', '025317', '305', '39744.0

['08/17', '120145', '1086', '4348', '-1667.0', '0', '7', '32']
['08/17', '120145', '1087', '5248', '-370.0', '0', '9', '49']
['08/17', '120145', '1088', '1727', '-1389.0', '0', '3', '7']
['08/17', '120145', '1089', '8664', '3889.0', '0', '12', '106']
['08/17', '120145', '1090', '4763', '-833.0', '0', '11', '42']
['08/17', '120145', '1091', '5356', '-1991.0', '0', '11', '42']
['08/17', '120010', '1092', '6652', '278.0', '28', '16', '--']
['08/17', '120010', '1093', '2861', '-1435.0', '6', '4', '--']
['08/17', '120010', '1094', '5987', '556.0', '27', '15', '--']
['08/17', '120010', '1095', '3482', '-1296.0', '7', '10', '--']
['08/17', '120010', '1096', '8788', '1481.0', '34', '37', '--']
['08/17', '120010', '1097', '5841', '509.0', '23', '16', '--']
['08/17', '120010', '1098', '8865', '1157.0', '39', '21', '--']
['08/17', '119996', '1099', '6797', '-880.0', '25', '11', '--']
['08/17', '119996', '1100', '2659', '417.0', '12', '7', '--']
['08/17', '119996', '1101', '9033', '2269.0', '45', 

['08/17', '120065', '1219', '196', '-231.0', '0', '0', '--']
['08/17', '120065', '1220', '2085', '-324.0', '6', '6', '--']
['08/17', '120065', '1221', '1386', '-1019.0', '2', '2', '--']
['08/17', '120088', '1222', '370', '-509.0', '0', '0', '--']
['08/17', '120088', '1223', '506', '-648.0', '0', '1', '--']
['08/17', '120088', '1224', '583', '-139.0', '2', '1', '1']
['08/17', '120093', '1225', '8481', '1528.0', '70', '0', '70']
['08/17', '120093', '1226', '3626', '3056.0', '46', '0', '46']
['08/17', '120093', '1227', '5327', '2917.0', '54', '0', '54']
['08/17', '120093', '1228', '1231', '-324.0', '7', '0', '7']
['08/17', '120093', '1229', '8053', '3889.0', '75', '0', '75']
['08/17', '120022', '1230', '951', '-602.0', '4', '2', '--']
['08/17', '120022', '1231', '985', '-278.0', '5', '3', '--']
['08/17', '120022', '1232', '5071', '880.0', '37', '19', '--']
['08/17', '120022', '1233', '523', '93.0', '4', '1', '--']
['08/17', '120022', '1234', '1772', '-1065.0', '8', '3', '--']
['08/17', '1

{'spreadsheetId': '1ERAWxxUmiu-EkKFI-_MWCbc0FnxPMEJDswAX2Uoaq_4',
 'updates': {'spreadsheetId': '1ERAWxxUmiu-EkKFI-_MWCbc0FnxPMEJDswAX2Uoaq_4',
  'updatedRange': 'S2RAW!F4256:M4282',
  'updatedRows': 27,
  'updatedColumns': 8,
  'updatedCells': 216}}